In [42]:
import torch
from torchvision import models

# export

In [43]:
x: torch.Tensor = torch.randn(1, 3, 224, 224).cuda()

In [44]:
model: torch.nn.Module = models.shufflenet_v2_x0_5(
    weights=models.ShuffleNet_V2_X0_5_Weights.DEFAULT
).cuda()

In [45]:
# half
half = True

In [46]:
# 动态batch
dynamic_batch = True

In [47]:
onnx_path = "./models/shufflenet_v2_x0_5"
if dynamic_batch:
    onnx_path += "-dynamic_batch"
if half:
    onnx_path += "-half"
    model: torch.nn.Module = model.half()
    x: torch.Tensor = x.half()
onnx_path += ".onnx"
onnx_path

'./models/shufflenet_v2_x0_5-dynamic_batch-half.onnx'

In [48]:
torch.onnx.export(
    model=model,
    args=x,
    opset_version=17,
    f=onnx_path,
    input_names=["images"],
    output_names=["classes"],
    dynamic_axes={"images": {0: "batch_size"}, "classes": {0: "batch_size"}}
    if dynamic_batch
    else None,
)

# onnxsim

In [49]:
import onnx
from onnxsim import simplify

In [50]:
model_ = onnx.load(onnx_path)

In [51]:
model__, ok = simplify(model_)
print(ok)

True


In [52]:
onnx.save(model__, onnx_path)

# onnxslim

In [53]:
import onnxslim

In [54]:
model__ = onnxslim.slim(model_)

In [55]:
onnx.save(model__, onnx_path)